In [ ]:
#Scratch Paper

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("kc_house_data.csv")
df.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [60]:
math_df = df.drop(['id', 'date', 'bedrooms', 'bathrooms',
       'sqft_lot', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'long', 'sqft_living15', 'sqft_lot15'], axis=1)
math_df.head()

,price,sqft_living,floors,lat
0,221900.0,1180,1.0,47.5112
1,538000.0,2570,2.0,47.7210
2,180000.0,770,1.0,47.7379
3,604000.0,1960,1.0,47.5208
4,510000.0,1680,1.0,47.6168


If you want to to stats, start finding the mean.  Mean = the sum of the items to be counted divided by the number of items to be counted.

In [61]:

def find_mean(df):
    column_means = []
    for idx, col in enumerate(df):
        mean = (np.sum(df[col]) / len(df[col]))
        column_means.append(mean)
    print(f' Mean: {column_means}')
    return column_means
find_mean(math_df)

 Mean: [540296.5735055795, 2080.3218502569803, 1.4940964022780943, 47.56009299439737]


[540296.5735055795, 2080.3218502569803, 1.4940964022780943, 47.56009299439737]

When you have the mean you can get the standard deviation, a basic number for how far away items are away from the mean.  Calculate this by the sumation of each variable subtrated by it's mean and divide that by number of items.  Find the square root of the result

In [65]:
def var_and_stddev(df):

    columns_std = []
    columns_variance = []
    for idx, col in enumerate(df):
        selection = 0
        mn = np.mean(df[col])
        selection += (df[col]-mn)**2
        selection /= len(df[col])
        variance = np.sum(selection)
        columns_std.append(np.sqrt(variance))
        columns_variance.append(variance)

    return columns_std, columns_variance
stddev(math_df)

([367359.634929548,
  918.0848694252896,
  0.5396702964403804,
  0.13854856047357553],
 [134953101375.5708,
  842879.8274676512,
  0.2912440288600481,
  0.019195703609300015])

In [63]:
math_df.describe()

,price,sqft_living,floors,lat
count,2.159700e+04,21597.000000,21597.000000,21597.000000
mean,5.402966e+05,2080.321850,1.494096,47.560093
std,3.673681e+05,918.106125,0.539683,0.138552
min,7.800000e+04,370.000000,1.000000,47.155900
25%,3.220000e+05,1430.000000,1.000000,47.471100
50%,4.500000e+05,1910.000000,1.500000,47.571800
75%,6.450000e+05,2550.000000,2.000000,47.678000
max,7.700000e+06,13540.000000,3.500000,47.777600


So far, so good, but we'd like to compare our predictor values to the price more directly.  Lets start with covariance.

In [ ]:
y = math_df.price
math_df = df.drop(['price'], axis=1)

In [92]:

'''
def covariance(x,y):
    covariance_list = []
    for idx, col in enumerate(x):
        covariance = (np.sum(x[col]-np.mean(x[col]))*(np.sum(y)-np.mean(y)))
        covariance /= len(math_df['sqft_living'])
        covariance_list.append(covariance)
    return covariance_list
covariance(math_df['sqft_living'],y)
'''

# covariance matrix =   ([[x,x], [x,y], 
#                        [y,x], [y,y]])
sqft = np.cov(math_df['sqft_living'],y)
flrs =  np.cov(math_df['floors'],y)
lat = np.cov(math_df['lat'],y)
print(f' square feet:\n {sqft} \n floors:\n {flrs} \n latitude:\n {lat}')
    
    

 square feet:
 [[8.42918857e+05 2.36744731e+08]
 [2.36744731e+08 1.34959350e+11]] 
 floors:
 [[2.91257515e-01 5.09144504e+04]
 [5.09144504e+04 1.34959350e+11]] 
 latitude:
 [[1.91965925e-02 1.56104871e+04]
 [1.56104871e+04 1.34959350e+11]]


What I wanted to do was to write a function but I can't get it.  So instead I'll be interpreting the covariance matrix resulting in running each predictor against the price.  Did I mention I'll still a student?

These numbers are all very nice but we have they are not scaled.  We really don't know what the 2.367 positive relation for square footage means against the 5.09 of the floors.  We *can* however, scale thes using correlation AKA the Pearson correlation coefficient.

r = n sigma xy - (sigma x)(sigma y) / the square root of (sigma of (x - mean of x)^2) (sigma of (y - mean of y)^2)

what's returned is a scaled version of covariance from -1 being a negative 1:1 relation, and 1 being a positive 1:1 relation.

In [93]:
np.corrcoef(math_df['sqft_living'],y)

array([[1.       , 0.7019173],
       [0.7019173, 1.       ]])

I'm being lazy, I know -- but look at that! These variables have a 1 corrceof with themselves (we know the method works!) and a whoping 0.7 with eachother.

but wait--we can use this to fill in a linreg equation.

1st
m = r (std_y / std_x)

2nd
b = y_mean (m * x_mean)

3rd
y_prime = b + m*x_mean

And look at that.  We now have a decent predictor of what each addition square foot will do to the price of a home. 

In [98]:
def slope_y_intercept_and_linreg(r,x,y):
    #this code is broken but my intent should be clear
    mu_x = []
    std_x = []
    for idx, col in enumerate(x):
        mu = np.mean(x[col])
        mu_x.append(mu)
        
    mu_y = np.mean(y)
    for idx, col in enumerate(x):
        std = np.std(x[col])
        std_x.append(std)
    y_std = np.std(y)
    slopes = [r * (std_x[i]/std_y) for i in mu_x]
    intercepts = [mu_y - (slopes[i]*mu_x[i]) for i in slopes and mu_x]
    linreg = [intercepts[i] + (slopes[i]*mu_x[i]) for i in intercepts and slopes and mu_x]
    return linreg

slope_y_intercept_and_linreg()

TypeError: list indices must be integers or slices, not numpy.float64

Now that we've fitted a linear regression it's about time we tested it's strength as a prediting model.  For that we'll need to calculate R^s or the  SSR by subracting the quotient of dividing SSE